## Zip to S3

This notebook automates the (re)build, install, zip, and upload to S3 required to update a AWS Layer.

In [1]:
from my_utils.os_utils import subprocess_execute, zipdir

ModuleNotFoundError: No module named 'my_utils'

In [4]:
# list of packages to build
packages_todo = [
    # list of tuples: ('package_name','from_pip')
    #('pandas',True),
    ('my_utils',False),
    ('my_dev',False),    
]
package_base_dir = '/home/vagrant/packages'
s3_path = 's3://andrewm4894-python-packages/latest/'
s3_region = 'us-west-2'
s3_bucket = s3_path.split('s3://')[1].split('/')[0]
s3_url = s3_path.replace('s3://','https://').replace(s3_bucket,f'{s3_bucket}.s3-{s3_region}.amazonaws.com')

In [7]:
# build and install locally each package
for package_name, from_pip in packages_todo:
    cmd = ''
    # run pip install within the python dir
    if from_pip:
        # if from pip then just use name
        cmd = cmd + f'pip install --upgrade --force-reinstall "{package_name}" -t {package_base_dir}/python/'
    else:
        # cd into the package dir
        cmd = f'cd {package_base_dir}/{package_name}'
        # run the build commands to create local egg file to be zipped up into an AWS layer
        cmd = cmd + f' | python setup.py bdist_egg'
        # cd into the 'python' dir which is to be zipped up and sent to AWS
        cmd = cmd + f' | cd {package_base_dir}/python'
        # otherwise point to local dir
        cmd = cmd + f' | pip install --upgrade "{package_base_dir}/{package_name}" -t {package_base_dir}/.'
    # kick off big command to do the stuff
    result = subprocess_execute(cmd)
    # look at results
    print(result)

... cmd to execute:
cd /home/vagrant/packages/my_utils | python setup.py bdist_egg | cd /home/vagrant/packages/python | pip install --upgrade "/home/vagrant/packages/my_utils" -t /home/vagrant/packages/.
... result:
Processing /home/vagrant/packages/my_utils
  Created wheel for my-utils: filename=my_utils-0.1-cp36-none-any.whl size=1789 sha256=bc07348d7495c1534fa6f9a38067e14bf9bfb1c0df9ddd187bcab007e8b53312
  Stored in directory: /tmp/pip-ephem-wheel-cache-lhfi8dgp/wheels/b0/a0/28/0e323ad727a3b13cc6aff0f961cf01b21fd665850f1d17450b
Successfully built my-utils

... cmd to execute:
cd /home/vagrant/packages/my_dev | python setup.py bdist_egg | cd /home/vagrant/packages/python | pip install --upgrade "/home/vagrant/packages/my_dev" -t /home/vagrant/packages/.
... result:
Processing /home/vagrant/packages/my_dev
  Created wheel for my-dev: filename=my_dev-0.1-cp36-none-any.whl size=1448 sha256=1fc430eb9ae0b81b10ba9a05bf0673a0cc08738d65be8b33c7f8997a8c887524
  Stored in directory: /tmp/pip-e

In [6]:
# make zip
zipdir('/home/vagrant/python/','python.zip')
# upload zip to s3 using aws cli
cmd = f'aws s3 cp --region {s3_region} {package_base_dir}/python.zip {s3_path}'
print(subprocess_execute(cmd))
print(f'... zip should be updated at: {s3_url}python.zip ...')

... cmd to execute:
aws s3 cp --region us-west-2 /home/vagrant/packages/python.zip s3://andrewm4894-python-packages/latest/
... result:
Error: failed to execute command: aws s3 cp --region us-west-2 /home/vagrant/packages/python.zip s3://andrewm4894-python-packages/latest/
b'/bin/sh: 1: aws: not found\n'

... zip should be updated at: https://andrewm4894-python-packages.s3-us-west-2.amazonaws.com/latest/python.zip ...
